In [116]:
import datetime
from time import sleep, time, perf_counter

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv
from pathlib import Path

from concurrent.futures import ThreadPoolExecutor, wait

In [132]:
# Парсим через pandas новости и рейтинги

def hacker_news(pages=10):

    news = pd.DataFrame(columns=['news', 'points'])

    def get_load_time(article_url):
        # будем ждать 3 секунды, иначе выводить exception и присваивать константное значение
        try:

            # делаем запрос по url статьи article_url
            response = requests.get(
                article_url, stream=True, timeout=3.000
            )
            # получаем время загрузки страницы
            load_time = response.elapsed.total_seconds()
        except Exception as e:
            print(e)
            load_time = ">3"
        return load_time

    #
    def get_info(page=1):
        url = f'https://news.ycombinator.com/?p={page}'

        responce = requests.get(url)

        # Получаем заголовок
        page = pd.read_html(responce.text)[2]
        page.drop([1], axis=1, inplace=True)
        page.rename(columns={0: 'index', 2: 'news'}, inplace=True)

        page = page[(~page['news'].isna()) & (page['news'] != 'More')]

        # Получаем рейтинг
        points = pd.DataFrame(
            page['news'][page['index'].isna()].reset_index(drop=True))
        points.rename(columns={'news': 'points'}, inplace=True)

        # Проверяем есть ли у новости рейтинг, если его нет то присваиваем ноль
        points['contains_points'] = points['points'].str.contains("points")
        points['points'] = points['points'].str.split(' ')

        def set_points(row):
            if row['contains_points']:
                return row[0][0]
            else:
                return 0

        points['points'] = points.apply(set_points, axis=1)

        # Прикручиваем рейтинг к новости
        news = pd.DataFrame(
            page['news'][~page['index'].isna()].reset_index(drop=True))

        result = pd.concat([news, points['points']], axis=1)

        soup = BeautifulSoup(responce.text, 'html.parser')

        # Начинаем собирать ссылки
        link_list = []

        for i in range(len(soup.find_all(class_='title'))):
            try:
                link = soup.find_all(class_='title')[i].find('a')['href']
                text = soup.find_all(class_='title')[i].text

                if text != 'More':
                    link_list.append([text, link])
                else:
                    pass
            except:
                pass
        
        trs = soup.find_all('tr', class_='athing')
        id_list = []

        for tr in trs:
            tr_id = tr['id']
            id_list.append(tr_id)

        id = pd.DataFrame(id_list, columns=['id'])

        links = pd.DataFrame(link_list)
        links = pd.concat([id, links], axis=1)

        links.rename(columns={0: 'news', 1: 'link'}, inplace=True)
        
        
        #links = pd.DataFrame(link_list)
        #links.rename(columns={0: 'news', 1: 'link'}, inplace=True)

        # Прикручиваем ссылки
        result = pd.merge(left=result, right=links)
        
        # Расчитываем время
        result['load_time'] = result['link'].apply(get_load_time)

        return result

    for i in range(pages):
        news = pd.concat([news, get_info(i+1)])

    news.reset_index(drop=True, inplace=True)
    
    

    return news

In [133]:
test = hacker_news(1)

HTTPSConnectionPool(host='macgui.com', port=443): Max retries exceeded with url: /news/article.php?t=493 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x147b96f90>: Failed to establish a new connection: [Errno 61] Connection refused'))
HTTPSConnectionPool(host='iknowwhatyoudownload.com', port=443): Read timed out. (read timeout=3.0)


In [134]:
test

,news,points,id,link,load_time
0,The coolest robot I've ever built (youtube.com),144,38162881,https://www.youtube.com/watch?v=bO-DWWFolPw,0.19159
1,Validating Data in Elixir: Using Ecto and Nimb...,7,38175413,https://blog.appsignal.com/2023/11/07/validati...,0.175681
2,Show HN: Patterns – Habit Tracker App (apps.ap...,53,38174614,https://apps.apple.com/ua/app/patterns-habit-t...,0.182601
3,PostgreSQL Encryption: The Available Options (...,98,38173141,https://www.hezmatt.org/~mpalmer/blog/2023/11/...,0.400674
4,Exploring the Macintosh ROM (2019) (macgui.com),24,38163037,https://macgui.com/news/article.php?t=493,>3
5,IBM Rebus (ibm.com),130,38160357,https://www.ibm.com/design/language/ibm-logos/...,0.692262
6,OneSignal (YC S11) Is Hiring Engineers (onesig...,0,38175863,https://onesignal.com/careers#section-job,0.142737
7,Strong evidence for new light isotope of nitro...,68,38161560,https://phys.org/news/2023-10-strong-evidence-...,0.47108
8,You don't need to work on hard problems (2020)...,118,38174128,https://www.benkuhn.net/hard/,0.134485
9,New models and developer products (openai.com),1026,38166420,https://openai.com/blog/new-models-and-develop...,0.396453


---

In [121]:
url = f'https://news.ycombinator.com/?p={1}'

In [122]:
response = requests.get(url)

In [123]:
soup = BeautifulSoup(response.text, 'html.parser')


In [136]:
soup.find_all('tr', class_='athing')[0].text

"\n1. The coolest robot I've ever built (youtube.com)"

In [125]:
trs = soup.find_all('tr', class_='athing')
id_list = []

for tr in trs:
    tr_id = tr['id']
    id_list.append(tr_id)
    
id_list

['38162881',
 '38175413',
 '38174614',
 '38173141',
 '38163037',
 '38160357',
 '38175863',
 '38161560',
 '38174128',
 '38166420',
 '38171589',
 '38169290',
 '38159762',
 '38161745',
 '38155899',
 '38161743',
 '38150333',
 '38171322',
 '38151299',
 '38172437',
 '38175311',
 '38175026',
 '38167363',
 '38174452',
 '38173457',
 '38158309',
 '38166098',
 '38171648',
 '38170980',
 '38167423']

In [131]:
link_list = []
for i in range(len(soup.find_all(class_='title'))):
    try:
        link = soup.find_all(class_='title')[i].find('a')['href']
        text = soup.find_all(class_='title')[i].text

        if text != 'More':
            link_list.append([text, link])
        else:
            pass
    except:
        pass

trs = soup.find_all('tr', class_='athing')
id_list = []

for tr in trs:
    tr_id = tr['id']
    id_list.append(tr_id)

id = pd.DataFrame(id_list, columns=['id'])

links = pd.DataFrame(link_list)
links = pd.concat([id, links], axis=1)

links.rename(columns={0: 'news', 1: 'link'}, inplace=True)
links

,id,news,link
0,38162881,The coolest robot I've ever built (youtube.com),https://www.youtube.com/watch?v=bO-DWWFolPw
1,38175413,Validating Data in Elixir: Using Ecto and Nimb...,https://blog.appsignal.com/2023/11/07/validati...
2,38174614,Show HN: Patterns – Habit Tracker App (apps.ap...,https://apps.apple.com/ua/app/patterns-habit-t...
3,38173141,PostgreSQL Encryption: The Available Options (...,https://www.hezmatt.org/~mpalmer/blog/2023/11/...
4,38163037,Exploring the Macintosh ROM (2019) (macgui.com),https://macgui.com/news/article.php?t=493
5,38160357,IBM Rebus (ibm.com),https://www.ibm.com/design/language/ibm-logos/...
6,38175863,OneSignal (YC S11) Is Hiring Engineers (onesig...,https://onesignal.com/careers#section-job
7,38161560,Strong evidence for new light isotope of nitro...,https://phys.org/news/2023-10-strong-evidence-...
8,38174128,You don't need to work on hard problems (2020)...,https://www.benkuhn.net/hard/
9,38166420,New models and developer products (openai.com),https://openai.com/blog/new-models-and-develop...
